Modified RapidInsights EDA - Completed <br/>
Choropleth map - Completed <br/>


2021-05-11 <br/>
US Broadband <br/>
https://github.com/rfordatascience/tidytuesday/blob/master/data/2021/2021-05-11/readme.md

In [ ]:

#imports and basic outlines

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

import seaborn as sns

from copy import deepcopy
%matplotlib inline

from collections import Counter



In [ ]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_colwidth', 200)
from itertools import product
from scipy.optimize import curve_fit
import warnings
warnings.filterwarnings("ignore")

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio

from IPython.display import HTML

In [ ]:
#pio.templates['plotly_dark'].layout.colorway = ('#0840D9','#882775','#00A34C','#B86200','#AA250E','#1D8682')

### Read the data

In [ ]:
df_broadband=pd.read_csv("broadband.csv")

In [ ]:
df_broadband.head()

In [ ]:
df_broadband_zip=pd.read_csv("broadband_zip.csv")

In [ ]:
df_broadband_zip.head()

In [ ]:
url = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'
geo = pd.read_json(url)
geo.head()

In [ ]:
geo.features[0]

In [ ]:
df_broadband[df_broadband['COUNTY ID']==01001]

In [ ]:
geo[geo['Name']=='New Jersey'].head()

### Preliminary Insights

In [ ]:
df_broadband.info()

In [ ]:
df_broadband_zip.info()

In [ ]:
sns.histplot(df_broadband, x="BROADBAND USAGE")

In [ ]:
sns.histplot(df_broadband_zip, x="BROADBAND USAGE")

In [ ]:
def check_duplicated(df):

    '''
    Helper function that highlights the duplicated values within the dataframe
    Arguments
    =========
    df: pandas dataframe

    Returns
    =======
    None 

    Shows the duplicated values present within the dataframe
    '''

    if len(df[df.duplicated()]) > 0:
        print("No. of duplicated entries: ", len(df[df.duplicated()]))
        print("### Duplicated values")
        print(df[df.duplicated(keep=False)].sort_values(by=list(df.columns)))
    else:
        print("No duplicated entries found")

In [ ]:
check_duplicated(df_broadband)

In [ ]:
check_duplicated(df_broadband_zip)

In [ ]:
def missing_values_table(df):

    '''
    Helper function that provides information on the rows that contain missing values within the dataframe

    Arguments
    =========
    df: pandas dataframe
    
    Returns
    =======
    mis_val_table_ren_columns: A dataframe that contains the columns with missing values and how many are missing 
                               compared to the entire dataframe  

    '''

    # Total missing values
    mis_val = df.isnull().sum()
    
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    
    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    
    # Print some summary information
    print("Your selected dataframe has " + str(df.shape[1]) + " columns and a total of "+str(df.shape[0])+" values\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
                    
    # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [ ]:
missing_values_table(df_broadband)

In [ ]:
missing_values_table(df_broadband_zip)

In [ ]:
def check_data_type(df):

    '''
    Helper function that shows the data type of the dataframe
    Arguments
    =========
    df: pandas dataframe
    
    Returns
    =======
    None

    Comments
    ========
    Shows the data type, the number of unique values and the first value (to validate the data type) of each column in the dataframe
    '''

    interesting= pd.DataFrame(df.dtypes,columns=["Data_Type"])
    interesting["First_value"]=df.iloc[0,:]
    unique_values= df.nunique()
    interesting["Unique_values"]=unique_values
    print(interesting)

In [ ]:
check_data_type(df_broadband)

In [ ]:
check_data_type(df_broadband_zip)

### Focusing on NJ

In [ ]:
df_broadband_NJ=df_broadband[df_broadband["ST"]=="NJ"]
df_broadband_zip_NJ=df_broadband_zip[df_broadband_zip["ST"]=="NJ"]


In [ ]:
df_broadband_NJ.head()

In [ ]:
df_broadband_zip_NJ.head()

In [ ]:
fig = px.choropleth_mapbox(df_broadband_NJ, geojason=counties, location='COUNTY NAME', color='BROADBAND USAGE',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, #center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'BROADBAND USAGE':'BROADBAND USAGE'}
                          )
fig.show()

In [ ]:
fig = px.choropleth(
    df_broadband_zip_NJ, 
    locations="COUNTY ID",                    
    color="BROADBAND USAGE",
    hover_name="COUNTY NAME", # column to add to hover information
    color_continuous_scale=px.colors.sequential.matter)
fig.show()

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
fig = px.choropleth_mapbox(df_broadband_NJ, geojson=counties, locations='COUNTY ID', color='BROADBAND USAGE',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5
                           #labels={'unemp':'unemployment rate'}
                          )
fig.show()

In [ ]:
import plotly.figure_factory as ff
#import plotly-geo
fig = ff.create_choropleth(fips=df_broadband_NJ["COUNTY ID"].astype(str), 
                           #scope=['NJ'],
                           values=df_broadband_NJ['BROADBAND USAGE'], 
                           title='NY Public Transit Use by County', 
                           legend_title='% Public Transit')
fig.layout.template = None
fig.show()

In [ ]:
df_broadband_zip_NJ.plot("BROADBAND USAGE")

In [ ]:
px.histogram(df_broadband_zip_NJ, x="BROADBAND USAGE")

In [ ]:
sns.histplot(df_broadband_zip_NJ, x="BROADBAND USAGE")

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

import plotly.express as px

fig = px.choropleth_mapbox(df, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()